In [ ]:
#Image classification

import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# Set device for training purpouse
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# setting paths
train_dir = 'D:/SolarGuard Intelligent Defect Detection/train'
valid_dir = 'D:/SolarGuard Intelligent Defect Detection/valid'

# Data Transforms with Augmentation for training
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Loading the Datasets
train_data = datasets.ImageFolder(root=train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(root=valid_dir, transform=valid_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

# Load Pretrained ResNet50 Model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {acc:.4f}")

# Validation Evaluation
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

val_acc = correct / total
print(f"\nValidation Accuracy: {val_acc:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=train_data.classes))


Epoch [1/10], Loss: 43.3101, Accuracy: 0.4915
Epoch [2/10], Loss: 25.1083, Accuracy: 0.7842
Epoch [3/10], Loss: 13.5394, Accuracy: 0.8576
Epoch [4/10], Loss: 8.1499, Accuracy: 0.9107
Epoch [5/10], Loss: 5.4372, Accuracy: 0.9480
Epoch [6/10], Loss: 4.3216, Accuracy: 0.9525
Epoch [7/10], Loss: 3.6155, Accuracy: 0.9616
Epoch [8/10], Loss: 2.4729, Accuracy: 0.9774
Epoch [9/10], Loss: 2.4896, Accuracy: 0.9684
Epoch [10/10], Loss: 2.7739, Accuracy: 0.9706

Validation Accuracy: 0.9921

Classification Report:
                   precision    recall  f1-score   support

        Bird-drop       0.99      1.00      0.99       207
            Clean       1.00      0.98      0.99       193
            Dusty       0.98      0.99      0.99       190
Electrical-damage       1.00      1.00      1.00       103
  Physical-Damage       0.99      0.99      0.99        69
     Snow-Covered       1.00      1.00      1.00       123

         accuracy                           0.99       885
        macro avg  

In [ ]:
#saving the training data
torch.save(model.state_dict(), "solar_classifier.pth")